Loading images, pruning saved model & quatizing pruned model:



In [1]:
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install matplotlib
%pip install numpy
%pip install scikit-learn
%pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Att

image loader file


In [5]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

#change with your own directory with the dataset while grading
main_dir = 'C://Users//joris//Downloads//dataset//dataset'

#converts training set to grayscale
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (32, 32))
    image = image / 255.0
    image = np.stack((image,) * 3, axis=-1)
    return image


#loads and labels all the images
def load_images_and_labels(main_dir):
    images = []
    labels = []

    for subdir in ['train', 'test', 'valid']:
        subdir_path = os.path.join(main_dir, subdir)
        image_filenames = [f for f in os.listdir(subdir_path) if f.endswith(".jpg") or f.endswith(".png")]

        for filename in sorted(image_filenames):
            image_path = os.path.join(subdir_path, filename)
            image = preprocess_image(image_path)
            images.append(image)
            labels.append(ord(filename[0].upper()) - ord('A'))

    return np.array(images), np.array(labels)



images, labels = load_images_and_labels(main_dir)


def visualize_images(images, num_images):
    plt.figure(figsize=(num_images * 4, 4))
    for i in range(num_images):
        ax = plt.subplot(2, num_images, i + 1)
        plt.imshow(images[i])
        plt.title("Original")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

x_combined, y_combined = load_images_and_labels(main_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'C://Users//tijnh//Documents//mod8//new_data//new_data/train'

shapes the images into train test and val

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import json
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Convert labels to numerical format
num_classes = 29
one_hot_labels = to_categorical(y_combined, num_classes=num_classes)

# Split the data
total_samples = x_combined.shape[0]
num_test = int(0.02 * total_samples)
num_val = int(0.01 * total_samples)
num_train = total_samples - num_test - num_val

x_train = x_combined[:num_train]
y_train = one_hot_labels[:num_train]

x_val = x_combined[num_train:num_train + num_val]
y_val = one_hot_labels[num_train:num_train + num_val]

x_test = x_combined[num_train + num_val:]
y_test = one_hot_labels[num_train + num_val:]

print(f"Training data shape: {x_train.shape}, {y_train.shape}")
print(f"Validation data shape: {x_val.shape}, {y_val.shape}")
print(f"Test data shape: {x_test.shape}, {y_test.shape}")


Training data shape: (1677, 32, 32, 3), (1677, 29)
Validation data shape: (17, 32, 32, 3), (17, 29)
Test data shape: (34, 32, 32, 3), (34, 29)


In [ ]:
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install matplotlib
%pip install numpy
%pip install scikit-learn
%pip install opencv-python

In [ ]:
!pip install tensorflow_model_optimization

Pruning code

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(29, activation='softmax'))

    return model

batch_size = 64

def pruning_parameters(sparsity, epochs):
    validation_split = 0.1  # 10% of the training set will be used for validation set.
    num_images = len(x_train) * (1 - validation_split)
    end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs
    # Set the parameters for pruning
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.01,
            final_sparsity=sparsity,
            begin_step=0,
            end_step=end_step
        )
    }
    return pruning_params

def pruning(sparsity_level, epochs, text):
    model = create_model()
    model.load_weights('final_model_weights.h5')  # Ensure weights are compatible with the model architecture

    # Apply pruning
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    pruning_params = pruning_parameters(sparsity_level, epochs)

    # Wrap the model with the pruning wrapper
    pruned_model = prune_low_magnitude(model, **pruning_params)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    pruned_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Define the pruning callbacks
    callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir='./logs')
    ]

    # Fit the model
    pruned_model.fit(x_train, y_train,
                     batch_size=batch_size,
                     epochs=epochs,
                     validation_split=0.1,
                     callbacks=callbacks)

    # Evaluate the model
    test_loss, test_accuracy = pruned_model.evaluate(x_test, y_test)
    print(f"Pruning {int(sparsity_level * 100)}% {text} - Accuracy: {test_accuracy:.4f}, Loss: {test_loss:.4f}")

    # Save the pruned model
    tf.saved_model.save(pruned_model, 'pruned_model')
    print("Pruned model saved to 'pruned_model'")

# Call the pruning function
pruning(0.5, 2, "with fine-tuning")  # Apply pruning with a target sparsity of 50% and 2 epochs for fine-tuning

Quantizing code

In [ ]:
import tensorflow as tf
import numpy as np

# Load the Keras model
#change with your own directory while grading
model_path = 'C://Users//joris//Downloads//final_pruned_model//final_pruned_model'

# Convert the model to a TensorFlow Lite model with quantization
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)

# Set the optimization flag for quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Optionally, provide a representative dataset for better quantization accuracy
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32))).batch(1).take(100):
        yield [input_value]

converter.representative_dataset = representative_data_gen

# Convert the model
tflite_model = converter.convert()

# Save the quantized model
# change with your own path while grading
quantized_model_path = 'C://Users//joris//Downloads//5_layered_model//quantized_5_layered_model.tflite'
with open(quantized_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Quantized model saved at {quantized_model_path}")


Quantized model saved at C://Users//joris//Downloads//5_layered_model//quantized_5_layered_model.tflite


Use the below unix command to covert the model from a ltfile file to a C array

xxd -i model_quantized2.tflite > model_data.cc